In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from tensorflow import keras
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.models import Sequential
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image
from keras.layers import Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import glob
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
import matplotlib.pyplot as plt
import cv2

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Display the data set of xray models.
training = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)


training_data = training.flow_from_directory(
        '/kaggle/input/chest-xray-pneumonia/chest_xray/train/',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=200,#Creates the batch size 
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels


def displayimages(numberofimages):
        for _ in range(numberofimages):
            img, label = training_data.next()
            print(img.shape)
            plt.imshow(img[0])
            plt.show()

            
displayimages(3)# Displays the first 100 training images.

In [ ]:


# Define the path to your dataset
dataset_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/train/'

# Initialize lists to store data and labels
X = []
y = []

# Create a list to store class labels
class_labels = []

# Iterate through the subdirectories and load the images and labels
for class_folder in os.listdir(dataset_dir):
    class_path = os.path.join(dataset_dir, class_folder)
    if os.path.isdir(class_path):
        # Load images from the class folder
        image_paths = glob.glob(os.path.join(class_path, '*.jpeg'))  # Assuming your images have .jpeg extension
        X.extend(image_paths)
        class_labels.extend([class_folder] * len(image_paths))

# Check the class distribution
class_distribution = Counter(class_labels)

# Create a bar chart for class distribution
plt.bar(class_distribution.keys(), class_distribution.values())
plt.xlabel('Class Labels')
plt.ylabel('Number of Samples')
plt.title('Class Distribution')
plt.xticks(rotation=45)
plt.show()

# Create a pie chart for class distribution
plt.pie(class_distribution.values(), labels=class_distribution.keys(), autopct='%1.1f%%')
plt.title('Class Distribution')
plt.show()




****Data set is very unbalanced ****

**Simple binary neural networ with drop out**

In [ ]:


input_shape = (150, 150, 3)
train_dir = "/kaggle/input/chest-xray-pneumonia/chest_xray/train/"
test_dir = "/kaggle/input/chest-xray-pneumonia/chest_xray/test/"

input_shape = (150, 150, 3)  # Adjust the dimensions as needed

# Create a binary neural network model
model = Sequential()

# Binary convolutional layers
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128,(3,3))),
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(512,(3,3))),
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(512, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# Flatten the output for fully connected layers
model.add(Flatten())




# Binary fully connected layers
model.add(Dense(128))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))   # Assuming binary classification

# Compile the model
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0001), metrics=['accuracy'])

# Data augmentation (you can customize these parameters)
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(150, 150), batch_size=32, class_mode='binary')
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(150, 150), batch_size=32, class_mode='binary')

# Train the model
history = model.fit(train_generator, steps_per_epoch=len(train_generator), epochs=12, validation_data=test_generator, validation_steps=len(test_generator))

# Evaluate the model
accuracy = model.evaluate(test_generator, steps=len(test_generator))[1]
print(f"Test accuracy: {accuracy}")



In [ ]:


model.save('chestxray.h5')#saves model 

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()


****Make prediction of are model ****

In [ ]:
img_path = "/kaggle/input/chest-xray-pneumonia/chest_xray/test/PNEUMONIA/person96_bacteria_465.jpeg"/kaggle/input/chest-xray-pneumonia/chest_xray/test/PNEUMONIA/person96_bacteria_465.jpeg
img = image.load_img(img_path, target_size=(150, 150))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)  # Add batch dimension
img = img / 255.0  # Rescale pixel values to [0, 1]

prediction = model.predict(img)
if prediction > 0.5:
    print("Pneumonia")
else:
    print("Normal")
